In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# 로컬 파일 시스템에서 로그 파일을 읽어옵니다.
log_file_path = '/home/rtcl/Documents/experiment/logs/receiver/receiver_log.txt'
with open(log_file_path, 'r') as file:
    receiver_log = file.read()

log_entries = []
for line in receiver_log.split('\n'):
    if line.startswith('Received at'):
        try:
            receive_time_str = line.split('Received at ')[1].split(':')[0].strip()
            send_time_str = line.split('Sent at ')[1].split(',')[0].strip()
            delay_str = line.split('Delay: ')[1].split(' ')[0].strip()
            priority = line.split('Priority: ')[1].strip()

            receive_time = float(receive_time_str)
            send_time = float(send_time_str)
            delay = float(delay_str)

            log_entries.append((receive_time, send_time, delay, priority))
        except Exception as e:
            print(f"Error processing line: '{line}', Error: {e}")

df_delays = pd.DataFrame(log_entries, columns=['Receive Time', 'Send Time', 'Delay', 'Priority'])
df_delays['Receive Time'] = pd.to_datetime(df_delays['Receive Time'], unit='s')
df_delays['Send Time'] = pd.to_datetime(df_delays['Send Time'], unit='s')

# 평균 지연 시간과 표준 편차를 계산합니다.
average_delays = df_delays.groupby('Priority')['Delay'].mean()
std_delays = df_delays.groupby('Priority')['Delay'].std()

# 데이터를 시각화합니다.
plt.figure(figsize=(15, 7))
for priority in df_delays['Priority'].unique():
    subset = df_delays[df_delays['Priority'] == priority]
    plt.plot(subset['Receive Time'], subset['Delay'], label=f'Delay {priority}')

# 평균 지연 시간과 표준 편차를 출력합니다.
for priority in average_delays.index:
    plt.axhline(y=average_delays[priority], color='grey', linestyle='--')
    plt.text(df_delays['Receive Time'].min(), average_delays[priority], f'{priority} Avg Delay: {average_delays[priority]:.2f} s, Std: {std_delays[priority]:.2f}', backgroundcolor='white')

plt.title('Delays over Time by Priority')
plt.xlabel('Receive Time')
plt.ylabel('Delay (seconds)')
plt.legend()
plt.show()


ModuleNotFoundError: No module named 'pandas'